In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gseapy
from scipy.stats import ttest_ind # importando uma biblioteca estatistica do scipy
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
sc.settings.verbosity = 3            
sc.logging.print_header()

In [ ]:
adata = sc.read_h5ad('Ziegler_final_2.h5ad') #inserir os dados com os nomes corretos para leitura
adata

# análise de qualidade

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')
qc = sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt' )
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 6000, :]
adata = adata[adata.obs.pct_counts_mt < 40, :]

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
adata

#                                            checkpoint 
# melhor salvar para não perder trabalho feito até aqui, então exporte os dados

In [ ]:
#adata.write('feminino_quality.h5ad')

In [ ]:
#adata = sc.read_h5ad('masculino_quality.h5ad') #usar somente se for reiniciar a análise


# Preprocessamento e escalonamento

In [ ]:
#normalizando por CPM
adata_CPM = adata.copy() 
adata_CPM.raw = adata_CPM 
sc.pp.normalize_total(adata_CPM,
                      target_sum =1e6)

In [ ]:
#sc.pp.pca(adata_CM)
#sc.pl.pca_overview(adata_CM, color='Cohort_Disease_WHO_Score')

In [ ]:
#not_malat1 = adata_CM.var.index != 'MALAT1' 
#adata_no_malat1 = adata_CM[:, not_malat1]

#sc.pp.pca(adata_no_malat1)
#sc.pl.pca_overview(adata_no_malat1, color='Cohort_Disease_WHO_Score')

In [ ]:
sc.pp.log1p(adata_CPM)

In [ ]:
sc.pp.highly_variable_genes(adata_CPM, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(adata_CPM)

In [ ]:
adata_CPM.raw = adata_CPM

In [ ]:
adata_CPM = adata_CPM[:, adata_CPM.var.highly_variable]

In [ ]:
sc.pp.regress_out(adata_CPM, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.scale(adata_CPM)


In [ ]:
sc.tl.pca(adata_CPM, svd_solver='arpack')
sc.pl.pca_overview(adata_CPM, color='Cohort_Disease_WHO_Score')

# Check point

In [ ]:
#adata_CPM.write('ziegler_final_CPM_normalizado.h5ad')

In [ ]:
#adata = sc.read_h5ad('masculino_normalizado.h5ad') #usar somente se for reiniciar a análise

In [ ]:
sc.tl.rank_genes_groups(adata_CPM, 'Cohort_Disease_WHO_Score',
                        groups=['COVID19_WHO_6-8'], 
                        reference='Ctrl_WHO_0', 
                        method='t-test',
                        key_added = 'WHO_6-8_vs_ctrl')

In [ ]:
adata_CPM

In [ ]:
mynames = [x[0] for x in adata_CPM.uns['WHO_6-8_vs_ctrl']['names'][:30]]
sc.pl.matrixplot(adata_CPM, mynames, groupby = 'Cohort_Disease_WHO_Score')# save = 'heat_Conval_vs_WHO_6-8.pdf')

# importando os DEGs

In [ ]:
tab_dif4 = sc.get.rank_genes_groups_df(adata_CPM, key = 'WHO_6-8_vs_ctrl', group= None) #cut of seleciona oas genes masi 

In [ ]:
tab_dif4

# checkpoint, usar apenas depois de realizar todos DEGS

In [ ]:
#adata_CPM.write('ziegler_final_CPM_normalizado.h5ad') # usar depois de realizar todas as comparações e adicionar uma key para salvar as estatisticas

# Exportando as tabelas de DEGs

In [ ]:
#exportando as tabelas DEGs
tab_dif4.to_csv("DEGS_feminino_Conval_vs_WHO_6-8.csv", sep=",")

In [ ]:
tab_dif4.to_excel("DEGS_feminino_Conval_vs_WHO_6-8.xlsx")

# plotando um grafico de violino para genes de interesse

In [ ]:
sc.pl.violin(adata_CPM, ['TMPRSS2'],
             groupby='sex',
             scale ='width',
             rotation =90,
             ylabel = 'Expressão média de $\it{TMPRSS2}$',
             stripplot =True,
             jitter= True,
             size =1,
             title = 'Variação da expressão de TMPRSS2 por genero',
             save ='TMPRSS2_genero.png')

In [ ]:
sc.pl.violin(adata_CPM, ['ACE2'], 
             groupby='sex',
             scale ='width',
             rotation =90,
             ylabel = 'Expressão média de $\it{ACE2}$',
             stripplot =True,
             jitter= True,
             size =1,
             title = 'Variação da expressão de ACE por genero',
             save ='ACE2_genero.png')

In [ ]:
sc.pl.violin(adata_CPM, ['AR'], 
             groupby='sex',
             scale ='width',
             rotation =90,
             ylabel = 'Expressão média de $\it{AR}$',
             stripplot =True,
             jitter= True,
             size =1,
             title = 'Variação da expressão de AR por genero',
             save ='AR_genero.png')

In [ ]:
markers =['TMPRSS2','ACE2','AR']

In [ ]:
sc.pl.stacked_violin(adata_CPM, markers,
                     groupby='sex',
                     log=False,
                     row_palette = 'hls',
                     standard_scale= 'var',
                     stripplot= True,
                     jitter= True,
                     size= 1,
                     swap_axes= False,
                     scale='width',
                     title= "Expressão dos genes $\it{TMPRSS2}$, $\it{ACE2}$ e $\it{AR}$ em pacientes do gênero \n masculino (M) e feminino (F)",
                     figsize= (10,10),
                     save= 'violino_masculino_todos_generos.png')

# heat map para os genes mais diferencialmente expressos entre os degs, usar os genes mais sig donw e sig up para gerar o mynames

In [ ]:
#mynames = [x[0] for x in adata_CPM.uns['Deg_WHO_1-5_vs_Ctrl']['names'][:10000]]
sc.pl.heatmap(adata_CPM, mynames, groupby = 'Cohort_Disease_WHO_Score')

#  daqui em diante, eve-se fazer o enrriquecimento de conjunto de genes